In [44]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

import numpy as np
import pandas as pd
import json

# Cases analysis

In this notebook, we will analyse the cases respect their similarity with other cases. To do that, we will use the files called `similarityMatrix_.txt`.

In [45]:
filePath = 'data/similarityMatrixWin.txt'

In [46]:
simDF = pd.read_csv(filePath, sep=";", header=None)
simDF.columns = ["Ci", "Cj", "Sim"]
simDF.head()

,Ci,Cj,Sim
0,C0-1,C0-1,1.000000
1,C0-1,C0-2,1.000000
2,C0-1,C25-6,0.471978
3,C0-1,C1-2,0.440476
4,C0-1,C1-4,0.373810


## Heatmap and histogram of similarity between cases

In [47]:
hist, bins = np.histogram(simDF[['Sim']],bins=50)

layout = go.Layout(
    title = 'Similarity histogram',
    xaxis = dict(title='Number of couple of cases'),
    yaxis = dict(title='similarity')
)

histSimValues = go.Figure(data = [go.Bar(x=bins,y=hist)], layout=layout)
iplot(histSimValues, filename='Histogram-similarity')

In [48]:
matrix = simDF.pivot(index='Ci', columns='Cj', values='Sim')

trace = go.Heatmap(z=simDF.pivot(index='Ci', columns='Cj', values='Sim').values,
                   x=matrix.columns,
                   y=matrix.index.values,
                   colorscale=[[0.0, 'rgb(255,255,255)'], [1.0, 'rgb(31, 119, 180)']])

data=[trace]
iplot(data, filename='labelled-heatmap')

## Distance Graph between cases

In [49]:
%load_ext py_d3

The py_d3 extension is already loaded. To reload it, use:
  %reload_ext py_d3


In [50]:
%%d3

<style>
.nodes circle {
  stroke: black	;
  stroke-width: 0px;
}
</style>

<svg width="900" height="600"></svg>
<script type="text/javascript" src="data/graphCasesWinElo.json"></script>

<script>

//create somewhere to put the force directed graph
var svg = d3.select("svg"),
    width = +svg.attr("width"),
    height = +svg.attr("height");
    
var radius = 15; 

var nodes_data = data.nodes;
var links_data = data.links;

//set up the simulation and add forces  
var simulation = d3.forceSimulation().nodes(nodes_data);
                              
var link_force =  d3.forceLink()
    .id(function(d) {return d.name;})
    .distance(function(d) { return (1 - d.similarity) * 1000; })
    .links(links_data);

simulation
    .force("link", link_force);
        
//add tick instructions: 
simulation.on("tick", tickActions );

//add encompassing group for the zoom 
var g = svg.append("g")
    .attr("class", "everything"); 

var color = d3.scaleSequential()
    .domain([0, 30])
    .interpolator(d3.interpolateRainbow)

//draw circles for the nodes 
var node = g.append("g")
        .attr("class", "nodes") 
        .selectAll("circle")
        .data(nodes_data)
        .enter()
        .append("circle")
        .attr("r", radius)
        .attr("fill", function(d) {
            return color(d.group);
        });

//add drag capabilities  
var drag_handler = d3.drag()
	.on("start", drag_start)
	.on("drag", drag_drag)
	.on("end", drag_end);	
	
drag_handler(node);


//add zoom capabilities 
var zoom_handler = d3.zoom()
    .on("zoom", zoom_actions);

zoom_handler(svg);     

/** Functions **/

function dist(d) {
    return 1 - d.similarity;
}

//Drag functions 
//d is the node 
function drag_start(d) {
 if (!d3.event.active) simulation.alphaTarget(0.3).restart();
    d.fx = d.x;
    d.fy = d.y;
}

//make sure you can't drag the circle outside the box
function drag_drag(d) {
  d.fx = d3.event.x;
  d.fy = d3.event.y;
}

function drag_end(d) {
  if (!d3.event.active) simulation.alphaTarget(0.3);
  d.fx = null;
  d.fy = null;
}

//Zoom functions 
function zoom_actions(){
    g.attr("transform", d3.event.transform)
}

function tickActions() {
   //update circle positions each tick of the simulation 
   node
    .attr("cx", function(d) { return d.x; })
    .attr("cy", function(d) { return d.y; });
} 
</script>